In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/cemb2020/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/cemb2020/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from fastai.text.all import *
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER, CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex
from collections import defaultdict

In [3]:
# tokenizer from https://stackoverflow.com/questions/58105967/spacy-tokenization-of-hyphenated-words
def custom_tokenizer(nlp):
    infixes = (
        LIST_ELLIPSES
        + LIST_ICONS
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
                al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
            ),
            r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
            #r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        ]
    )

    infix_re = compile_infix_regex(infixes)

    return Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)

In [4]:
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = custom_tokenizer(nlp) # need custom tokenizer cus spacy nlp pipe doesn't properly tokenize hyphenated words

In [5]:
data = "../data/1000examples.csv"
df = pd.read_csv(data) # Note: do not need to tokenize text for putting through model because it will autotokenize it

FileNotFoundError: [Errno 2] No such file or directory: '../data/1000examples.csv'

## Create Adversarial Examples

In [ ]:
# sourced from https://github.com/JHL-HUST/PWWS/blob/dfd9f6b9a8fb9a57e3a0b9207e1dcbe8a624618c/paraphrase.py
def get_pos(word):
    '''Wordnet POS tag'''
    pos = word.tag_[0].lower()
    if pos in ['r', 'n', 'v']:  # adv, noun, verb
        return pos
    elif pos == 'j':
        return 'a'  # adj

In [ ]:
def evaluate_word_saliency(sentence_list, pos_neg):  
    word_saliency_array = []
    # Compute word saliency (difference in classification accuracy if word replace with <unk>)
    for i, word in enumerate(sentence_list):
        copy_list = sentence_list.copy()
        copy_list[i] = "<UNK>"
        new_sentence = " ".join(copy_list)
        x_i = learn.predict(new_sentence)[2][pos_neg]
        word_saliency = x - x_i
        word_saliency_array.append((word, word_saliency))
    return word_saliency_array

In [68]:
def find_best_synonym(sentence_list, index, word, x, pos_neg):
    """Should return synonym to replace it and the change in classification probability that it causes"""
    
    # https://www.askpython.com/python/examples/pos-tagging-in-nlp-using-spacy
    synonyms = defaultdict(list)

    token = word.text
    part_of_speech = get_pos(word)

    # create a list of synonyms of the word based on part of speech
    for syn in wordnet.synsets(token, pos=part_of_speech):
        for i in syn.lemmas():
            synonyms[token].append(i.name())

    # go through all the synonyms and assess how much they change classification
    max_synonym = ""
    max_prob_difference = 0
    for s in synonyms[token]:
        copy_list = sentence_list.copy()
        copy_list[index] = s
        new_sentence = " ".join(copy_list)
        x_hat_i = learn.predict(new_sentence)[2][pos_neg]
        prob_difference = (x - x_hat_i).item()
        if (prob_difference > max_prob_difference):
            max_prob_difference = prob_difference
            max_synonym = s

    return (max_synonym, max_prob_difference)    

In [69]:
# defined in Eq.(8)
def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [70]:
def calculate_score(word_saliency, prob_difference):
    return word_saliency * prob_difference

In [73]:
model_path = "models/model2500_40000_0.pkl"
learn = load_learner(model_path, cpu=False)
sentence = "The movie was absolutely stupendous! The actors were so well-spoken."
sentence_list = word_tokenize(sentence)
pos_neg = 1

# x -> the base probility of the correct classification of the sentence
initial_prediction = learn.predict(sentence)
x = initial_prediction[2][pos_neg] # second index depends if pos or neg example 0 -> prob of neg, 1 -> prob of pos

# find word saliency of all words
word_saliency_array = evaluate_word_saliency(sentence_list, pos_neg)
word_saliency_vector = np.array([pair[1] for pair in word_saliency_array])
word_saliency_vector = softmax(word_saliency_vector)


# spacy piping
sentence = nlp(sentence)

word_scores =  []

for index, word in enumerate(sentence):
    word_saliency = word_saliency_vector[index]
    
    # Get set of synonyms from wordnet (if NE get similar NEs)
    # return value: (synonym to replace the word with, probability change caused by replacement)
    synonym_pair = find_best_synonym(sentence_list, index, word, x, pos_neg)
    synonym = synonym_pair[0]
    prob_difference = synonym_pair[1]
    
    if (prob_difference > 0):
        # use word saliency and change in probability to calculate the score 
        score = calculate_score(word_saliency, prob_difference)
        word_scores.append((index, synonym, score))
    
# order word replacements by score
word_scores.sort(key = itemgetter(2), reverse=True)

['The', 'movie', 'was', 'absolutely', 'stupendous', '!', 'The', 'actors', 'were', 'so', 'well-spoken', '.']
[0.10150255 0.10278237 0.07349627 0.08022118 0.08952869 0.0753866
 0.07310504 0.07731725 0.0735458  0.08213419 0.09913504 0.07184502]
The
movie


was


absolutely


stupendous


!
The
actors


were


so


well-spoken


.
[(3, 'dead', 0.0788502709418708), (2, 'cost', 0.07208647738567908), (8, 'cost', 0.05573321539178444), (1, 'moving-picture_show', 0.006459675671635523), (4, 'colossal', 0.005798435874464936), (9, 'thusly', 0.0050663955975807085), (7, 'worker', 0.003982102932315712)]


In [74]:
# replace words in sentence until the prediction is false
new_list = sentence_list.copy()
adversarial_sentence = ""
for (word_index, synonym, _) in word_scores:
    new_list[word_index] = synonym
    new_sentence = " ".join(new_list)
    new_prediction = learn.predict(new_sentence)
    print(new_prediction[0])
    if (initial_prediction[0] != new_prediction[0]):
        adversarial_sentence = new_sentence
        print("success")
        break
print(adversarial_sentence)

neg
success
The movie was dead stupendous ! The actors were so well-spoken .
